Zadanie 1 (6 pk)
Utwórz stronę dash, która:
*	Wczyta dane z pliku winequality.csv do tabeli pandas i je wyświetli jako tabelę (10 wierszy).
*	Będzie pozwalała na wybór modelu regresji lub klasyfikacji (dropdown lista albo radio button albo jeszcze coś innego – wybrać sobie).
*	Na podstawie wyboru z pkt. b i funkcji zwrotnej zapewni reakcję łańcuchową. W przypadku wyboru regresji ma pokazywać wykres z plotly zależności pH od wybranej przez użytkownika zmiennej, a w przypadku wyboru klasyfikacji ma pokazywać zależność target (red/white) od wybranej przez użytkownika innej zmiennej. Zastanowić się, który wykres będzie to najlepiej obrazował.


In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

In [38]:
df = pd.read_csv('winequelity.csv', index_col = 0)

In [55]:
df.sample(20).sort_values(by='target')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,target
1241,9.8,0.370,0.39,2.50,0.079,28.0,65.0,0.99729,3.16,0.59,9.8,5,red
753,8.3,0.650,0.10,2.90,0.089,17.0,40.0,0.99803,3.29,0.55,9.5,5,red
683,8.1,0.780,0.23,2.60,0.059,5.0,15.0,0.99700,3.37,0.56,11.3,5,red
918,8.4,0.360,0.32,2.20,0.081,32.0,79.0,0.99640,3.30,0.72,11.0,6,red
1069,8.0,0.620,0.35,2.80,0.086,28.0,52.0,0.99700,3.31,0.62,10.8,5,red
259,10.0,0.310,0.47,2.60,0.085,14.0,33.0,0.99965,3.36,0.80,10.5,7,red
361,8.6,0.450,0.31,2.60,0.086,21.0,50.0,0.99820,3.37,0.91,9.9,6,red
777,6.9,0.635,0.17,2.40,0.241,6.0,18.0,0.99610,3.40,0.59,10.3,6,red
4299,6.3,0.280,0.30,6.60,0.208,60.0,154.0,0.99478,3.10,0.40,9.4,6,white
2044,5.7,0.385,0.04,12.60,0.034,22.0,115.0,0.99640,3.28,0.63,9.9,6,white


In [40]:
def generate_table(df, r=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in df.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Th(df.iloc[i][col]) for col in df.columns
            ]) for i in range(min(len(df), r))
        ])
    ])

In [69]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Dash praca domowa"),
    generate_table(df),
    html.Label(['Wybierz wykres:'],style={'font-weight': 'bold'}),
    dcc.RadioItems(
        id='radio',
        options=[
            {'label': 'regresja', 'value': 'graph1'},
            {'label': 'klasyfikacja', 'value': 'graph2'},
        ]
    ),
    html.Div(
        dcc.Graph(id='graph', style={'height': 400, 'width': 800}),
    )
])

@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='radio', component_property='value')]
)

def build_graph(value):
    if value == 'graph1':
        return px.scatter(
            df,
            x="pH",
            y="fixed acidity",
            opacity=0.90)

    else:
        return px.scatter(
            # taki wykres fajniejszy i więcej pokazuje niż wykres target od czegoś jednego
            df,
            x="density",
            y="total sulfur dioxide",
            color='target',
            opacity=0.90)


In [70]:
if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/
